# Data Cleaning and Merging

### Get CSVs to DataFrames

In [2]:
#Import Dependacies
import pandas as pd
import os

In [3]:
#Get CSVs and Store in DataFrames

#Get Paths
airPath = os.path.join('..', 'Resources', 'air_pollution.csv')
co2Path = os.path.join('..', 'Resources', 'co_emissions.csv')
elePath = os.path.join('..', 'Resources', 'electricity_production.csv')
resPath = os.path.join('..', 'Resources', 'respiratory_disease_deaths.csv')

#Load DataFrames
airDF = pd.read_csv(airPath)
co2DF = pd.read_csv(co2Path)
eleDF = pd.read_csv(elePath)
resDF = pd.read_csv(resPath)

### Clean DataFrames to Desires Columns

In [4]:
#Air Quality
airDF = airDF[['Location', 'SpatialDimValueCode', 'Period', 'FactValueNumeric']]
airDF = airDF.rename(columns={'SpatialDimValueCode':'Country Code', 'Location':'Country Name','Period':'Year','FactValueNumeric':'Particulate Matter'})
airDF = airDF.groupby(['Country Name', 'Year'])['Particulate Matter'].mean()
#airDF = airDF.sort_values(by='Country Code')
airDF.head(100)

Country Name  Year
Afghanistan   2010    75.274
              2011    72.958
              2012    74.534
              2013    78.902
              2014    74.396
                       ...  
Austria       2015    13.980
              2016    12.572
              2017    12.600
              2018    12.964
              2019    11.768
Name: Particulate Matter, Length: 100, dtype: float64

In [5]:
#CO2 Levels
co2DF = co2DF.rename(columns={'Entity':'Country Name', 'Code': 'Country Code', 'Annual CO₂ emissions (zero filled)':'CO2 Emissions'})
co2DF = co2DF.loc[co2DF['Year'] >= 2010]
co2DF.head()

,Country Name,Country Code,Year,CO2 Emissions
260,Afghanistan,AFG,2010,8364803.5
261,Afghanistan,AFG,2011,11838316.0
262,Afghanistan,AFG,2012,10035314.0
263,Afghanistan,AFG,2013,9250510.0
264,Afghanistan,AFG,2014,9170309.0


In [6]:
#Electricity Production
eleDF = eleDF[['COUNTRY', 'YEAR', 'MONTH', 'PRODUCT', 'yearToDate', 'share']]
eleDF = eleDF.loc[eleDF['MONTH'] == 12]
eleDF = eleDF.drop(columns=['MONTH'])
eleDF = eleDF.rename(columns={'COUNTRY':'Country Name', 'YEAR':'Year', 'PRODUCT':'Fuel Type', 'yearToDate': 'Value', 'share':'Share'})
eleDF = eleDF.loc[(eleDF['Fuel Type'] == 'Renewables') | (eleDF['Fuel Type'] == 'Non-renewables')]

renewablesDF = eleDF.loc[eleDF['Fuel Type'] == 'Renewables']
nonrenewablesDF = eleDF.loc[eleDF['Fuel Type'] == 'Non-renewables']

eleDF['Renewables Value'] = renewablesDF['Value']
eleDF['Renewables Share'] = renewablesDF['Share']
eleDF['Non-Renewables Value'] = nonrenewablesDF['Value']
eleDF['Non-Renewables Share'] = nonrenewablesDF['Share']

eleDF = eleDF.drop(columns=['Value', 'Share'])
eleDF = eleDF.groupby(['Country Name', 'Year']).mean()
eleDF = eleDF.reset_index()
eleDF.head()

,Country Name,Year,Renewables Value,Renewables Share,Non-Renewables Value,Non-Renewables Share
0,Argentina,2015,33567.573000,0.247105,104023.260000,0.752895
1,Argentina,2016,31013.883000,0.213672,108158.284000,0.786328
2,Argentina,2017,33292.217000,0.249992,104623.013000,0.750008
3,Argentina,2018,34652.219560,0.324108,104535.897342,0.675892
4,Argentina,2019,34524.081521,0.275338,97938.231521,0.724662


In [7]:
#Respiratory Deaths
resDF = resDF.rename(columns={'Entity':'Country Name', 'Code':'Country Code', 'Deaths - Chronic respiratory diseases - Sex: Both - Age: Age-standardized (Rate)':'Respiratory Death Rate'})
resDF = resDF.loc[resDF['Year'] >= 2010]
resDF.head()

,Country Name,Country Code,Year,Respiratory Death Rate
20,Afghanistan,AFG,2010,81.60
21,Afghanistan,AFG,2011,79.38
22,Afghanistan,AFG,2012,77.27
23,Afghanistan,AFG,2013,75.24
24,Afghanistan,AFG,2014,73.60


### Merge DataFrames

In [8]:
#Initial Merge
mergedDF = pd.merge(airDF, co2DF, on=['Country Name', 'Year'], how='inner')
mergedDF = pd.merge(mergedDF, resDF)
mergedDF = pd.merge(mergedDF, eleDF)
mergedDF.head()

,Country Name,Year,Particulate Matter,Country Code,CO2 Emissions,Respiratory Death Rate,Renewables Value,Renewables Share,Non-Renewables Value,Non-Renewables Share
0,Argentina,2015,13.940,ARG,191743840.0,34.86,33567.573000,0.247105,104023.260000,0.752895
1,Argentina,2016,13.690,ARG,189920430.0,35.81,31013.883000,0.213672,108158.284000,0.786328
2,Argentina,2017,13.008,ARG,186898880.0,35.94,33292.217000,0.249992,104623.013000,0.750008
3,Argentina,2018,12.384,ARG,180599330.0,35.09,34652.219560,0.324108,104535.897342,0.675892
4,Argentina,2019,11.832,ARG,178511620.0,34.85,34524.081521,0.275338,97938.231521,0.724662


In [9]:
#Remove Unnecessary Columns and Reorganize
mergedDF = mergedDF.drop(columns='Country Code')
mergedDF = mergedDF[['Country Name', 'Year', 'Renewables Value', 'Renewables Share', 'Non-Renewables Value', 'Non-Renewables Share', 'CO2 Emissions', 'Particulate Matter', 'Respiratory Death Rate']]
mergedDF.head()

,Country Name,Year,Renewables Value,Renewables Share,Non-Renewables Value,Non-Renewables Share,CO2 Emissions,Particulate Matter,Respiratory Death Rate
0,Argentina,2015,33567.573000,0.247105,104023.260000,0.752895,191743840.0,13.940,34.86
1,Argentina,2016,31013.883000,0.213672,108158.284000,0.786328,189920430.0,13.690,35.81
2,Argentina,2017,33292.217000,0.249992,104623.013000,0.750008,186898880.0,13.008,35.94
3,Argentina,2018,34652.219560,0.324108,104535.897342,0.675892,180599330.0,12.384,35.09
4,Argentina,2019,34524.081521,0.275338,97938.231521,0.724662,178511620.0,11.832,34.85


In [10]:
mergedDF.sort_values("Particulate Matter").head(10)

,Country Name,Year,Renewables Value,Renewables Share,Non-Renewables Value,Non-Renewables Share,CO2 Emissions,Particulate Matter,Respiratory Death Rate
108,Finland,2017,30673.984000,0.466033,34369.203000,0.533967,44577796.0,5.770,14.82
107,Finland,2016,29614.757000,0.425184,36588.477000,0.574816,47181860.0,5.818,14.89
110,Finland,2019,30976.840200,0.483117,35075.638202,0.516883,42381840.0,5.834,14.60
157,Iceland,2016,18168.608000,0.999789,3.000000,0.000211,3485184.0,5.938,17.69
98,Estonia,2017,1689.635000,0.176163,9558.855000,0.823837,18762488.0,5.944,10.65
109,Finland,2018,31229.926874,0.420427,36302.741120,0.579573,45796460.0,5.974,14.69
158,Iceland,2017,18844.210000,0.999971,2.085000,0.000029,3601620.0,6.002,17.95
327,Sweden,2016,88471.466000,0.550305,64059.097000,0.449695,43415984.0,6.024,18.47
160,Iceland,2019,19117.132001,0.999226,2.661000,0.000774,3546263.0,6.034,17.50
328,Sweden,2017,94168.450000,0.574028,66357.546000,0.425972,42704240.0,6.072,18.51


In [11]:
#Save as new CSV
#outPath = os.path.join('..', 'Outputs', 'cleaned_data.csv')
#mergedDF.to_csv(outPath)